In [27]:
# NF‑κB Luciferase Reporter Assay – pylabrobot version
# Author: Guangxin Zhang 
# This script automates the luciferase activity measurement protocol using
# pylabrobot and the custom backend you created earlier today.
from pylabrobot.resources.opentrons import OTDeck
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.opentrons_backend_jump import OpentronsBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources import Coordinate
from pylabrobot.resources.opentrons import (
corning_96_wellplate_360ul_flat,
    nest_12_reservoir_15ml,
    nest_1_reservoir_195ml,
    opentrons_96_tiprack_300ul
)

# ──────────────────────────────────────
# User‑configurable constants (µL)
MEDIUM_VOL = 100     # volume of spent medium to remove
PBS_VOL    = 50      # PBS wash volume
LYSIS_VOL  = 30      # lysis buffer volume
LUC_VOL    = 100     # luciferase reagent volume
TOTAL_COL  = 12      # process A1–A12

# ──────────────────────────────────────
def _build_deck(lh: LiquidHandler):
    """Load all labware on the deck and return handy shortcuts."""
    # Tip‑racks on slots 8, 11, 1, 4  (order preserved to match Opentrons layout)
    tiprack_slots = [8, 11, 1, 4]
    tipracks = []
    # Load tip racks
    for slot_i in tiprack_slots:
      tr = opentrons_96_tiprack_300ul(name=f"tiprack_{slot_i}")
      lh.deck.assign_child_at_slot(tr, slot=slot_i)
      tipracks.append(tr)
    # Working 96‑well plate at slot 6
    working_plate = corning_96_wellplate_360ul_flat(name="working_plate")
    lh.deck.assign_child_at_slot(working_plate, slot=6)

    # 12‑channel reservoir (PBS, Lysis, Luciferase) at slot 3
    reagent_stock = nest_12_reservoir_15ml(name="reagent_stock")
    lh.deck.assign_child_at_slot(reagent_stock, slot=3)

    # 1‑channel waste reservoir at slot 9
    waste_res = nest_1_reservoir_195ml(name="waste_res")
    lh.deck.assign_child_at_slot(waste_res, slot=9)

    return {
        "tip_racks"     : tipracks,
        "working_plate" : working_plate,
        "reagent_res"   : reagent_stock,
        "waste_res"     : waste_res
    }

In [28]:
def _tip_gen(tip_racks):
    """Yield the next available tip."""
    for rack in tip_racks:
        for tip in rack:
            yield tip
    raise RuntimeError("Out of tips!")

In [29]:
backend = OpentronsBackend(host="localhost", simulation=True)
lh = LiquidHandler(backend=backend, deck=OTDeck())
await lh.setup()
vis = Visualizer(resource=lh)
await vis.setup()
deck = _build_deck(lh)
tips = _tip_gen(deck["tip_racks"])

Websocket server started at http://127.0.0.1:2126
File server started at http://127.0.0.1:1342 . Open this URL in your browser.


In [30]:
pbs        = deck["reagent_res"][0]
lysis      = deck["reagent_res"][1]
luciferase = deck["reagent_res"][2]
waste      = deck["waste_res"][0]
wells_name = [f"A{i}" for i in range(1, 13)]
cells_all  = deck["working_plate"][wells_name] # A1–A12

In [31]:
# ────────── 1. Remove spent medium ──────────
for cell in cells_all:
    tip = next(tips)
    await lh.pick_up_tips(tip)
    await lh.aspirate([cell], [MEDIUM_VOL * 1.2], flow_rates=[0.2])
    await lh.dispense(waste, [MEDIUM_VOL * 1.4], flow_rates=[3])
    await lh.discard_tips()

In [32]:
await lh.pick_up_tips(next(tips))

AttributeError: 'OpentronsBackend' object has no attribute 'make_asp'

In [37]:


for cell in cells_all:


  await lh.aspirate(pbs, vols=[PBS_VOL], flow_rate = [3], blow_out_air_volume = [20])

  await lh.dispense([cell], [PBS_VOL + 20], flow_rate=0.3,
                offset=Coordinate(z=-2))

await lh.discard_tips()


/Users/guangxinzhang/Documents/Deep Potential/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:338: UserWarning: Extra arguments to backend.aspirate: {'flow_rate'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")
/Users/guangxinzhang/Documents/Deep Potential/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:338: UserWarning: Extra arguments to backend.dispense: {'offset', 'flow_rate'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")
